<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/issue/icg_input%E8%B7%9F%E7%B3%BB%E7%B5%B1%E8%BC%B8%E5%87%BA%E4%B8%8D%E8%83%BD%E5%AE%8C%E7%BE%8E%E4%B8%80%E8%87%B4%E7%9A%84%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt

fatal: destination path 'c620' already exists and is not an empty directory.


# import 

In [2]:
from c620.F import F
from c620.config import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


# functions

In [3]:
def check_two_data(d1,d2):
  if np.allclose(d1.values,d2.values):
    print('一樣')
  else:
    print('不一樣')

# demo data

In [4]:
tag = 'test001'
demo = joblib.load('./data/demo(real_data).pkl')
print(demo.keys())

icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']

icg_input

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c660_op', 'c670_op'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
2020-09-05,179.571663,1.1500,10.9300,36.900,12.342497,2.4600,73.490,19.220,62.680651,0.4920,37.0867,53.7119,88.480,748.0,0.0,3.0,41.111207
2021-02-16,182.702744,2.0556,12.1420,37.828,18.193422,2.8398,67.888,21.521,93.629074,0.6960,43.2400,42.5320,87.491,861.0,0.0,10.0,43.173817
2020-10-21,140.768653,0.7438,4.9644,68.043,17.639095,2.5231,74.979,16.199,67.150949,0.8400,55.7900,26.9700,84.901,809.0,0.0,2.0,44.523722
2021-01-20,181.033157,1.4680,11.8790,37.263,19.130981,1.9880,68.749,20.500,100.350426,0.6510,46.1770,40.4170,85.165,809.0,0.0,63.0,40.970806
2020-12-04,166.107956,2.1510,12.0470,37.287,19.038383,3.4740,65.729,19.557,99.873688,0.6486,43.8780,42.6020,86.102,805.0,0.0,35.0,39.059292


In [5]:
idx1 = icg_input.index[0]
idx2 = icg_input.index[1]

# 1.確保給定不同的輸入 會有不同的輸出

In [6]:
f = F(config)
f.Recommended_mode = True #(True or False)
f.real_data_mode = True

# 輸入1
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])

# 輸入2
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2 = f(icg_input.loc[[idx2]],c620_feed.loc[[idx2]],t651_feed.loc[[idx2]])

current Distillate Rate_m3/hr:0.23077957112638564 NA in Benzene_ppmw:980.0
current Distillate Rate_m3/hr:0.20156970242883998 NA in Benzene_ppmw:980.0


In [7]:
check_two_data(c620_wt1,c620_wt2)
check_two_data(c620_op1,c620_op2)
check_two_data(c660_wt1,c660_wt2)
check_two_data(c660_op1,c660_op2)
check_two_data(c670_wt1,c670_wt2)
check_two_data(c670_op1,c670_op2)

不一樣
不一樣
不一樣
不一樣
不一樣
不一樣


# 2.確保變更模式 推薦 和 試算 模式 有不同的輸出

In [8]:
# 試算模式
f.Recommended_mode = False #(True or False)
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])

# 推薦模式
f.Recommended_mode = True #(True or False)
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])

current Distillate Rate_m3/hr:0.23077957112638564 NA in Benzene_ppmw:980.0


In [9]:
check_two_data(c620_wt1,c620_wt2)
check_two_data(c620_op1,c620_op2)
check_two_data(c660_wt1,c660_wt2)
check_two_data(c660_op1,c660_op2)
check_two_data(c670_wt1,c670_wt2)
check_two_data(c670_op1,c670_op2)

不一樣
不一樣
不一樣
不一樣
不一樣
不一樣


# 確認icg_input 對應到後續系統的輸出值一致 主要有兩項要一致
* 1. benzene in c620 side
* 2. na in benzene

In [10]:
# 真實數據 試算模式
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
f.Recommended_mode = False #(True or False)
f.real_data_mode = True
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
naidx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]
c660_wt1.filter(regex='Sidedraw').iloc[:,naidx].sum(axis=1)*10000

            Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
2020-09-05                                              88.48               
            Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
2020-09-05                                              748.0     
            Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
2020-09-05                                          89.205144                                   


2020-09-05    469.522107
dtype: float64

In [11]:
# 真實數據 推薦模式
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
f.Recommended_mode = True #(True or False)
f.real_data_mode = True
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
naidx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]
c660_wt1.filter(regex='Sidedraw').iloc[:,naidx].sum(axis=1)*10000

            Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
2020-09-05                                              88.48               
            Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
2020-09-05                                              748.0     
current Distillate Rate_m3/hr:0.23077957112638564 NA in Benzene_ppmw:980.0
            Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
2020-09-05                                           80.77302                                   


2020-09-05    488.373393
dtype: float64

In [12]:
# 模擬數據 試算模式
demo = joblib.load('./data/demo.pkl')
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
idx1 = icg_input.index[0]
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
f.Recommended_mode = False #(True or False)
f.real_data_mode = False
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
naidx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]
c660_wt1.filter(regex='Sidedraw').iloc[:,naidx].sum(axis=1)*10000

         Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
124-020                                               70.0               
         Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
124-020                                              980.0     
         Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
124-020                                          79.807054                                   


124-020    833.028942
dtype: float64

In [13]:
# 模擬數據 推薦模式
demo = joblib.load('./data/demo.pkl')
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
idx1 = icg_input.index[0]
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
f.Recommended_mode = True #(True or False)
f.real_data_mode = False
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
naidx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]
c660_wt1.filter(regex='Sidedraw').iloc[:,naidx].sum(axis=1)*10000

         Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
124-020                                               70.0               
         Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
124-020                                              980.0     
current Distillate Rate_m3/hr:0.4746916636938635 NA in Benzene_ppmw:980.0
         Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
124-020                                          79.736179                                   


124-020    833.032422
dtype: float64